# <font color='black'> Практическое занятие 2 </font>
## <font color='black'> Анализ панельных данных: часть 1. Модели с фиксированными эффектами (FE-models) и модели со случайными эффектами (RE-models) </font>
Типичные панельные данные имеют как пространственное, так и временное измерение. К примеру, наблюдения для одной страны представлены за несколько лет. При такой структуре данных наивно было бы полагать, что условие о независимости наблюдений соблюдается. Наблюдения "внутри" одной пространственной единицы, разумеется, связаны. В этом случае мы не можем применять классическую линейную регрессию. В качестве альтернативы выступают широко применяемые модели с фиксированными эффектами. Реже в исследованиях применяются модели со случайными эффектами (вспомните, чем обусловлено их меньшее распространение). На практическом занятии 2 мы рассмотрим и тот, и другой класс моделей. 
Кратко о данных, с которыми мы сегодня будем работать:
* country -  страна: массив содержит данные по 27 посткоммунистическим странам
* period - период: данные усреднены по пятилеткам начиная с 1989 г. 
* fh_polity - индекс демократии, полученный на основе показателей Freedom House, Polity. Непрерывная шкала от 0 до 10, где 10 означает максимальное значение уровня демократии. 
* state_capacity - индекс государственной состоятельности. Приведен к непрерывной шкале от 0 до 10, где 10 - наиболее высокое значение государственной состоятельности. 

Подгрузим необходимые библиотеки и откроем массив "panel.dta". 

In [ ]:
import pandas as pd
import statsmodels.formula.api as statf 
import numpy as np
import numpy.linalg as la
import seaborn as sb 
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from scipy import stats

In [ ]:
data = pd.read_stata("panel.dta")
data.head(10)

В нашей первой модели мы учтем разные "стартовые" условия. Для этого добавим дамми-переменные на страну. Проинтерпретируйте оценки model1. Почему в данном случае мы получаем большой R-squared? 

In [ ]:
model1 = statf.ols(formula = 'fh_polity ~ state_capacity +C(country)', data = data).fit()
print(model1.summary())

Как Вы уже могли заметить выше, Python по умолчанию в качестве базовой категории (относительно которой происходит сравнение) выбирает первую пространственную единицу. При этом не всегда удобно интерпретировать относительно первой категории. Ниже рассмотрим, как можно изменить базовую категорию. Проинтерпретируйте оценки model1_2. 

In [ ]:
data['country1'] = pd.Categorical(data.country)
model1_2 = statf.ols(formula = 'fh_polity ~ state_capacity +C(country1, Treatment("Russia"))', data = data).fit()
print(model1_2.summary())

Проиллюстрируем результаты моделирования, предварительно сохранив предсказанные значения. Как мы видим, график отражает разные "стартовые" условия и одинаковый наклон (одинаковый характер взаимосвязи демократии и государственной состоятельности) в соответствии с оцененными моделями. 

In [ ]:
data['ypredicted'] = model1.predict()
data.head(10)

In [ ]:
sb.lmplot("state_capacity", "ypredicted", hue="country", data=data, palette="Set1", size = 7)

На следующем шаге оценим модель с внутригрупповым преобразованием. Вспомните, в чем ее отличие от LSDV-модели (модели с дамми-переменными), почему часто используется именно такая спецификация с внутригрупповым преобразованием. Обратите внимание на то, что для того, чтобы в Python оценить FE-модели и RE-модели, нужно предварительно указать, какая переменная показывает пространственное измерение, а какая - временное. 
* Проинтерпретируйте оценки model2. 
* Проинтерпретируйте значения F-test for Poolability. 

In [ ]:
data1 = data.set_index(['country', 'period'])
model2 = PanelOLS.from_formula('fh_polity ~ state_capacity + EntityEffects', data = data1).fit()
print(model2)

Для сравнения оценим модель со случайными эффектами. 
* Вспомните допущения данной модели и оцените критически 
* Какой метод оценивания используется в данном случае?

In [ ]:
model3 = RandomEffects.from_formula('fh_polity ~ 1 + state_capacity', data = data1).fit()
print(model3)

Применим тест Хаусмана для проверки гипотезы об отсутствии корреляции между индивидуальными эффектами и предикторами. В Python отсутствует автоматическая функция реализации данного теста, поэтому зададим в явном виде статистику для проверки гипотезы и ее распределение. 
* Проинтерпретируйте результаты тестирования 
* Каковы ограничения данного теста?

In [ ]:
b_fe = model2.params
b_re = model3.params
b_re_new = np.asarray(b_re)
b_re_new = np.delete(b_re_new, (0), axis = 0)

cov_fe = model2.cov
cov_re = model3.cov
cov_re = np.asarray(model3.cov)
cov_re
cov_re_new = np.delete(np.delete(cov_re, 0, axis = 0), 0, 1)
hausman = (b_fe - b_re_new).T.dot(la.inv(np.asmatrix(cov_fe - cov_re_new)).dot(b_fe - b_re_new))
df = b_fe.size
pvalue = stats.chi2.sf(hausman, df)
print('Chi-squared:', hausman, '\n' 'P-value:', pvalue, '\n' 'Degrees of freedom:', df)

Проверим, действительно ли FE-модель хорошо объясняет зависимую переменную. Оценим корреляцию зависимой переменной (наблюдаемого отклика) и предсказанного значения отклика по подгруппам. Если корреляция низкая (по модулю), то это говорит о том, что мы плохо объясняем моделью то, что происходит в данной группе (стране). Переоценим модель на подвыборке тех стран, для которых корреляцию по модулю больше 0.3 и посмотрим, сохранились ли наши результаты.  

In [ ]:
groups_data = pd.DataFrame(data.groupby('country')['ypredicted'].corr(data['fh_polity']))
def grouping(series): 
    if abs(series) < 0.3: return 0 
    elif abs(series) >= 0.3: return 1

groups_data['group'] = groups_data['ypredicted'].apply(grouping)
data_merged = pd.merge(data, groups_data, left_on = 'country', right_index=True)
data_merged.rename(columns = {"ypredicted_y":"corr"})

In [ ]:
high = data_merged[data_merged['group']==1]
low = data_merged[data_merged['group']==0]
model5 = statf.ols(formula = 'fh_polity ~ state_capacity + C(country)', data = high).fit()
print(model5.summary())